In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!nvidia-smi

Tue Jul 19 14:57:19 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2199.998
BogoMIPS:            4399.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

In [4]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q "datasets==2.2.2" seqeval
!pip install gradio
!sudo apt install tesseract-ocr
!pip install pytesseract

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# !pip freeze

In [6]:
!rm -rf 'find -type d -name .ipynb_checkpoints'

In [7]:
!rmdir /content/dataset/.ipynb_checkpoints
!rmdir /content/dataset-300dpi/test/annotation_dir/.ipynb_checkpoints
!rmdir /content/dataset/train/annotation_dir/.ipynb_checkpoints
!rmdir /content/dataset-300dpi/test/img_dir/.ipynb_checkpoints


!rmdir /content/dataset/test/.ipynb_checkpoints
!rmdir /content/dataset/train/.ipynb_checkpoints

rmdir: failed to remove '/content/dataset/.ipynb_checkpoints': No such file or directory
rmdir: failed to remove '/content/dataset-300dpi/test/annotation_dir/.ipynb_checkpoints': No such file or directory
rmdir: failed to remove '/content/dataset/train/annotation_dir/.ipynb_checkpoints': No such file or directory
rmdir: failed to remove '/content/dataset-300dpi/test/img_dir/.ipynb_checkpoints': No such file or directory
rmdir: failed to remove '/content/dataset/test/.ipynb_checkpoints': No such file or directory
rmdir: failed to remove '/content/dataset/train/.ipynb_checkpoints': No such file or directory


In [8]:
from datasets import load_dataset 
# from PIL import Image
# this dataset uses the new Image feature :)
dataset = load_dataset('/content/dataset.py')

Generating train split: 0 examples [00:00, ? examples/s]

dataset/train/img_dir/1.jpg
dataset/train/img_dir/2.jpg


Generating test split: 0 examples [00:00, ? examples/s]

dataset/test/img_dir/3.jpg
Dataset sroie downloaded and prepared to /root/.cache/huggingface/datasets/sroie/discharge/1.0.0/2076851cd758c9b2b04db99d2a68d752674c1fcdf3ac071e8786492d9b8b7092. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
dataset["train"].features

{'bboxes': Sequence(feature=Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None), length=-1, id=None),
 'id': Value(dtype='string', id=None),
 'image_path': Value(dtype='string', id=None),
 'ner_tags': Sequence(feature=ClassLabel(num_classes=12, names=['others', 'shipper', 'consignee', 'notify', 'receipt', 'portofloading', 'finaldestination', 'payable', 'discharge', 'description', 'weight', 'volume'], id=None), length=-1, id=None),
 'words': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [10]:
example = dataset["train"][1]
example["image_path"]

'dataset/train/img_dir/2.jpg'

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 2
    })
    test: Dataset({
        features: ['id', 'words', 'bboxes', 'ner_tags', 'image_path'],
        num_rows: 1
    })
})

In [12]:
# example = dataset["train"][0]
# example["image"]

In [13]:
words, bboxes, ner_tags = example["words"], example["bboxes"], example["ner_tags"]
print(words)
print(bboxes)
print(ner_tags)

['nan', 'nan', 'nan', 'nan', 'SEDE', 'FILIALE', 'DI', 'RAVENNA', 'FILIALE', 'DI', 'TORINO', 'SEDE', 'ESTERA', 'KOPER', 'nan', 'nan', 'nan', ' ', 'nan', 'nan', 'nan', 'Via', 'Fabio', 'Filzi,', '31-57122', 'Livorno', 'Via', 'G.', 'Rossi,', '70', 'Via', 'Valentino', 'Carrera,', '157', 'Ankaranska', '5', 'nan', 'nan', 'Tel.', '586', '896486-211862-211900', '48100', 'Ravenna', '10146', 'Torino', 'SI-6000', 'KOPER', '(SLOVENUIA)', 'nan', 'nan', 'Telefax', '586', '886517', 'Tel.', '0544/3547', '212316', 'Tel.', '011/4275865', 'Tel.', '386', '5', '6638950/51', 'es', 'nan', 'nan', 'e-mail:', 'base@basespa.com', 'Telefax:', '0544/216465', 'Mob.', '349', '9806103', 'Fax', '386', '5', '6638952', 'X', 'nan', 'FILIALE', 'DI', 'VENEZIA', 'FILIALE', 'DI', 'LA', 'SPEZIA', 'nan', 'Molo', 'A', 'fabbricato', '441', 'Via', 'Palmaria,', '8', '4', 'DNV', 'GL', 'ISO', '9001', 'nan', '30170', 'Porto', 'Marghera', '19126', 'La', 'Spezia', 'B', '|', 'R', 'fae', 'Company', 'Certified', 'nan', 'nan', '.', 'Tel.', 

In [14]:
from transformers import AutoProcessor

# we'll use the Auto API here - it will load LayoutLMv3Processor behind the scenes,
# based on the checkpoint we provide from the hub
processor = AutoProcessor.from_pretrained("microsoft/layoutlmv3-base", apply_ocr=False)

In [15]:
from datasets.features import ClassLabel

features = dataset["train"].features
column_names = dataset["train"].column_names
image_column_name = "image_path"
text_column_name = "words"
boxes_column_name = "bboxes"
label_column_name = "ner_tags"

# In the event the labels are not a `Sequence[ClassLabel]`, we will need to go through the dataset to get the
# unique labels.
def get_label_list(labels):
    unique_labels = set()
    for label in labels:
        unique_labels = unique_labels | set(label)
    label_list = list(unique_labels)
    label_list.sort()
    return label_list

if isinstance(features[label_column_name].feature, ClassLabel):
    label_list = features[label_column_name].feature.names
    # No need to convert the labels since they are already ints.
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
else:
    label_list = get_label_list(dataset["train"][label_column_name])
    id2label = {k: v for k,v in enumerate(label_list)}
    label2id = {v: k for k,v in enumerate(label_list)}
num_labels = len(label_list)

In [16]:
print(label_list)

['others', 'shipper', 'consignee', 'notify', 'receipt', 'portofloading', 'finaldestination', 'payable', 'discharge', 'description', 'weight', 'volume']


In [17]:
print(id2label)

{0: 'others', 1: 'shipper', 2: 'consignee', 3: 'notify', 4: 'receipt', 5: 'portofloading', 6: 'finaldestination', 7: 'payable', 8: 'discharge', 9: 'description', 10: 'weight', 11: 'volume'}


In [18]:
from PIL import Image

def prepare_examples(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']] #Image.open(examples[image_column_name])
  words = examples[text_column_name]
  boxes = examples[boxes_column_name]
  word_labels = examples[label_column_name]

  encoding = processor(images, words, boxes=boxes, word_labels=word_labels,
                       truncation=True, padding="max_length")

  return encoding

In [19]:
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

# we need to define custom features for `set_format` (used later on) to work properly
features = Features({
    'pixel_values': Array3D(dtype="float32", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=label_list)),
})

train_dataset = dataset["train"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)
eval_dataset = dataset["test"].map(
    prepare_examples,
    batched=True,
    remove_columns=column_names,
    features=features,
)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [20]:
train_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 2
})

In [21]:
example = train_dataset[0]
processor.tokenizer.decode(example["input_ids"])

'<s> nan nan nan nan COSTACURTA S.P.A-VICO nan nan nan VIA GRAZIOLI 30 nan 20161 MILANO nan nan nan HYUPJIN MACHINERY CO., LTD nan 601 1SA 16 EMTIBEUI 28-RO nan SIHEUNG-SI GYEONGGI-DO nan nan nan SOUTH KOREA nan nan nan HYUPJIN MACHINERY CO., LTD nan 601 1SA 16 EMTIBEUI 28-RO nan SIHEUNG-SI GYEONGGI-DO nan nan nan SOUTH KOREA nan nan nan UNAYZAH 06-Nov nan nan nan BUSAN nan nan nan Marks : nan nan nan HYUPJIN MACHINERY nan co., LTD nan nan 601, 1SA, 16, EMTIBEUI nan 28-RO, SIHEUNG-SI, nan GYEONGGI-DO nan REPUBLIC OF SOUTH nan KOREA nan nan nan GENOVA nan nan nan S.T.C. nan nan nan 1 CASE KGS 645 nan nan nan VICO-FLEX -103.1569343 nan nan nan CONVEYOR BELT nan nan nan FREIGHR PREPAID nan nan nan EXPRESS HBL nan nan nan Booking nan nan nan Gil91/2ANRERR nan nan nan SHIPPER’S LOAD, STOW AND COUNT. nan SHIPPER RESPONSIBLE FOR STOWING, nan PACKING AND ADEQUATELY LASHING OF CARGO. nan INTO CONTAINER(S) FOR SEA TRANSPORTATION. nan HOUSE/HOUSE nan FCL/FCL nan nan nan KGS nan nan nan PREPAID EX

In [22]:
train_dataset.set_format("torch")

In [23]:
import torch

example = train_dataset[0]
for k,v in example.items():
    print(k,v.shape)

pixel_values torch.Size([3, 224, 224])
input_ids torch.Size([512])
attention_mask torch.Size([512])
bbox torch.Size([512, 4])
labels torch.Size([512])


In [24]:
eval_dataset

Dataset({
    features: ['pixel_values', 'input_ids', 'attention_mask', 'bbox', 'labels'],
    num_rows: 1
})

In [25]:
eval_dataset.features['bbox']

Array2D(shape=(512, 4), dtype='int64', id=None)

In [26]:
processor.tokenizer.decode(eval_dataset[0]["input_ids"])

"<s> nan nan nan nan TUVIA nan nan nan ISTRUZIONI PER EMIS nan nan nan SAN MARTINO BUON AL nan nan nan *** BOOKI nan nan nan Shipper nan nan SIRMAN SPA nan nan VIALE DELL' INDUSTRI nan 35010 PIEVE DI CURT. nan ITALY nan nan nan Consignee nan ALLIED METALS INC. nan nan nan 2000 GOVERNOR HALIL nan 3023 MUZON CIT nan PHILIPPINES nan nan Place of receipt nan nan nan Ocean vessel nan OOCL FRANCE nan Port of discharge nan nan nan MANILA, PHILIPPINES nan Marks and numbers nan nan nan ALLIED METALS INC. nan nan nan Freight amount nan nan nan Delivery agent nan nan nan Spett. nan SIONE POLIZZA nan nan nan B, 07-10-2021 nan nan nan NG SI/21/306142 *** nan nan nan Nostri riferimenti 18/1300444 1 nan nan nan A, 09-Nov nan 'AROLO (PD) nan nan nan Notify nan SAME AS CNEE nan nan nan I HIGHWAY nan Y OF SAN JOSE DE nan nan nan Port of loading nan GENOVA nan Place of delivery nan nan nan Number and type of kind Description of goods Gross wght Weight Kg nan nan nan SAID TO CONTAIN nan 1 PALLET KGS. 185,

In [27]:
for id, label in zip(train_dataset[0]["input_ids"], train_dataset[0]["labels"]):
  print(processor.tokenizer.decode([id]), label.item())

<s> -100
 nan 0
 nan 0
 nan 0
 nan 0
 CO 1
ST -100
AC -100
UR -100
TA -100
 S 1
. -100
P -100
. -100
A -100
- -100
V -100
ICO -100
 nan 1
 nan 1
 nan 1
 V 1
IA -100
 G 1
RA -100
ZI -100
OL -100
I -100
 30 1
 nan 1
 2016 1
1 -100
 MIL 1
AN -100
O -100
 nan 0
 nan 0
 nan 0
 HY 2
UP -100
J -100
IN -100
 M 2
ACH -100
IN -100
ERY -100
 CO 2
., -100
 LTD 2
 nan 2
 601 2
 1 2
SA -100
 16 2
 E 2
MT -100
IB -100
E -100
UI -100
 28 2
- -100
RO -100
 nan 2
 SI 2
HE -100
UN -100
G -100
- -100
SI -100
 G 2
YE -100
ON -100
GG -100
I -100
- -100
DO -100
 nan 2
 nan 2
 nan 2
 SOU 2
TH -100
 K 2
ORE -100
A -100
 nan 0
 nan 0
 nan 0
 HY 3
UP -100
J -100
IN -100
 M 3
ACH -100
IN -100
ERY -100
 CO 3
., -100
 LTD 3
 nan 3
 601 3
 1 3
SA -100
 16 3
 E 3
MT -100
IB -100
E -100
UI -100
 28 3
- -100
RO -100
 nan 3
 SI 3
HE -100
UN -100
G -100
- -100
SI -100
 G 3
YE -100
ON -100
GG -100
I -100
- -100
DO -100
 nan 3
 nan 3
 nan 3
 SOU 3
TH -100
 K 3
ORE -100
A -100
 nan 0
 nan 0
 nan 0
 U 4
NA -100
Y -100
Z -100

In [28]:
from datasets import load_metric

metric = load_metric("seqeval")

In [29]:
import numpy as np

return_entity_level_metrics = False

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    if return_entity_level_metrics:
        # Unpack nested dictionaries
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

In [30]:
from transformers import LayoutLMv3ForTokenClassification

model = LayoutLMv3ForTokenClassification.from_pretrained("microsoft/layoutlmv3-base",
                                                         id2label=id2label,
                                                         label2id=label2id)

Some weights of LayoutLMv3ForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [31]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="layoutlmv3-finetuned-discharge",
                                  max_steps=1000,
                                  per_device_train_batch_size=5,
                                  per_device_eval_batch_size=5,
                                  learning_rate=1e-5,
                                  evaluation_strategy="steps",
                                  eval_steps=100,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  no_cuda=False)

In [32]:
from transformers.data.data_collator import default_data_collator

# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    data_collator=default_data_collator,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 2
  Num Epochs = 1000
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 1000
/usr/local/lib/python3.7/dist-packages/transformers/modeling_utils.py:703: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  "The `device` argument is deprecated and will be removed in v5 of Transformers.", FutureWarning


Step,Training Loss,Validation Loss


In [ ]:
trainer.evaluate()

In [ ]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained("/content/layoutlmv3-finetuned-discharge/checkpoint-500")#/content/layoutlmv3-finetuned-invoice/checkpoint-2000

In [ ]:
example = dataset["test"][0]
print(example.keys())

image = Image.open(example["image_path"])
words = example["words"]
boxes = example["bboxes"]
word_labels = example["ner_tags"]

encoding = processor(image, words, boxes=boxes, word_labels=word_labels, return_tensors="pt", truncation=True)
for k,v in encoding.items():
  print(k,v.shape)

In [ ]:
words

In [ ]:
with torch.no_grad():
  outputs = model(**encoding)

logits = outputs.logits
print(logits.shape)

predictions = logits.argmax(-1).squeeze().tolist()
print(predictions)

In [ ]:
labels = encoding.labels.squeeze().tolist()
print(labels)

In [ ]:
def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]

token_boxes = encoding.bbox.squeeze().tolist()
width, height = image.size

true_predictions = [model.config.id2label[pred] for pred, label in zip(predictions, labels) if label != - 100]
true_labels = [model.config.id2label[label] for prediction, label in zip(predictions, labels) if label != -100]
true_boxes = [unnormalize_box(box, width, height) for box, label in zip(token_boxes, labels) if label != -100]

In [ ]:
true_labels

## prediction Value

In [ ]:
from PIL import ImageDraw, ImageFont

draw = ImageDraw.Draw(image)

font = ImageFont.load_default()

def iob_to_label(label):
    #label = label[2:]
    if not label:
      return 'other'
    return label


label2color = {
    "others": 'white',
    "shipper": 'blue',
    "consignee": 'pink',
    "notify": 'orange',
    "receipt": 'purple',
    "portofloading": 'green',
    "finaldestination": 'maroon',
    "payable": 'black',
    "discharge": 'lime',
    "description": 'teal',
    "weight": 'aqua',
    "volume": 'gray'
  }

# label2color = {
#     "others": 'yellow',
#     "key": 'blue',
#     "value": 'red',
#   }

for prediction, box in zip(true_predictions, true_boxes):
    predicted_label = iob_to_label(prediction)
    # print(predicted_label)
    draw.rectangle(box, outline=label2color[predicted_label])
    draw.text((box[0] + 10, box[1] - 10), text=predicted_label, fill=label2color[predicted_label], font=font)

image

In [ ]:
# pr=[]
# for i,j in zip(predictions, words):
#   if id2label[i] =="produttore_key":
#     pr.append(j)
#   # print(i, j, ": " ,id2label[i])

# " ".join(pr)

## Ground Truth Value

In [ ]:
image = Image.open(example["image_path"]).convert("RGB")

draw = ImageDraw.Draw(image)

for word, box, label in zip(example['words'], example['bboxes'], example['ner_tags']):
  actual_label = iob_to_label(id2label[label])
  box = unnormalize_box(box, width, height)
  draw.rectangle(box, outline=label2color[actual_label], width=2)
  draw.text((box[0] + 10, box[1] - 10), actual_label, fill=label2color[actual_label], font=font)

image

### Inference

In [ ]:
# !tesseract — version

In [ ]:
import os
os.system('pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu')

import os
import PIL
import torch
import pandas as pd
import numpy as np
import gradio as gr
from tqdm import tqdm
from difflib import SequenceMatcher
from itertools import groupby
from datasets import load_metric
from datasets import load_dataset
from datasets.features import ClassLabel
from transformers import AutoProcessor
from PIL import Image, ImageDraw, ImageFont
from transformers import AutoModelForTokenClassification
from transformers.data.data_collator import default_data_collator
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D
from transformers import LayoutLMv3ForTokenClassification,LayoutLMv3FeatureExtractor

# define id2label
id2label={0: 'others', 1: 'produttore_key', 2: 'produttore_value', 3: 'cliente_key', 4: 'cliente_value', 5: 'unitloc_key', 6: 'unitloc_value', 7: 'operatore_key', 8: 'operatore_value', 9: 'referente_key', 10: 'referente_value', 11: 'cfproduttore_key', 12: 'cfproduttore_value', 13: 'telefono_key', 14: 'telefono_value', 15: 'emailcliente_key', 16: 'emailcliente_value', 17: 'datarichiesta_key', 18: 'datarichiesta_value', 19: 'orariorichiesta_key', 20: 'orariorichiesta_value', 21: 'emailproduttore_key', 22: 'emailproduttore_value', 23: 'mattina_key', 24: 'mattina_value', 25: 'pomeriggio_key', 26: 'pomeriggio_value', 27: 'cer_key', 28: 'cer_value', 29: 'descrizione_key', 30: 'descrizione_value', 31: 'sf_key', 32: 'sf_value', 33: 'classpericolo_key', 34: 'classpericolo_value', 35: 'destino_key', 36: 'destino_value', 37: 'confezionamento_key', 38: 'confezionamento_value', 39: 'destinazione_key', 40: 'destinazione_value'}

label_ints = np.random.randint(0,len(PIL.ImageColor.colormap.items()),42)
label_color_pil = [k for k,_ in PIL.ImageColor.colormap.items()]
label_color = [label_color_pil[i] for i in label_ints]
label2color = {}
for k,v in id2label.items():
  if v[:2] == '':
    label2color['o']=label_color[k]
  else:
    label2color[v[2:]]=label_color[k]

processor = AutoProcessor.from_pretrained("/content/layoutlmv3-finetuned-discharge/checkpoint-1000", apply_ocr=True)
model = AutoModelForTokenClassification.from_pretrained("/content/layoutlmv3-finetuned-discharge/checkpoint-1000")


def unnormalize_box(bbox, width, height):
     return [
         width * (bbox[0] / 1000),
         height * (bbox[1] / 1000),
         width * (bbox[2] / 1000),
         height * (bbox[3] / 1000),
     ]


def iob_to_label(label):
    return label


def process_image(image):
    width, height = image.size

    feature_extractor = LayoutLMv3FeatureExtractor(apply_ocr=True)
    encoding_feature_extractor = feature_extractor(image, return_tensors="pt")
    words, boxes = encoding_feature_extractor.words, encoding_feature_extractor.boxes

    custom_config = r'--oem 3 --psm 6'
    # encode
    encoding = processor(image, truncation=True, return_offsets_mapping=True, return_tensors="pt")
    offset_mapping = encoding.pop('offset_mapping')

    # forward pass
    outputs = model(**encoding)

    # get predictions
    predictions = outputs.logits.argmax(-1).squeeze().tolist()
    token_boxes = encoding.bbox.squeeze().tolist()

    # only keep non-subword predictions
    is_subword = np.array(offset_mapping.squeeze().tolist())[:,0] != 0
    true_predictions = [id2label[pred] for idx, pred in enumerate(predictions) if not is_subword[idx]]
    true_boxes = [unnormalize_box(box, width, height) for idx, box in enumerate(token_boxes) if not is_subword[idx]]

    true_boxes = true_boxes[1:-1]
    true_predictions = true_predictions[1:-1]

    preds = []
    l_words = []
    bboxes = []

    for i,j in enumerate(true_predictions):

      if j != 'O':
        preds.append(true_predictions[i])
        l_words.append(words[0][i])
        bboxes.append(true_boxes[i])

    return bboxes, preds, l_words, image



def visualize_image(final_bbox, final_preds, l_words, image):

      draw = ImageDraw.Draw(image)
      font = ImageFont.load_default()

      label2color = {
    "others": 'white',
    "shipper": 'blue',
    "consignee": 'pink',
    "notify": 'orange',
    "receipt": 'purple',
    "portofloading": 'green',
    "finaldestination": 'maroon',
    "payable": 'black',
    "discharge": 'lime',
    "description": 'teal',
    "weight": 'aqua',
    "volume": 'gray'
  }

      l2l ={
    "others": 'white',
    "shipper": 'blue',
    "consignee": 'pink',
    "notify": 'orange',
    "receipt": 'purple',
    "portofloading": 'green',
    "finaldestination": 'maroon',
    "payable": 'black',
    "discharge": 'lime',
    "description": 'teal',
    "weight": 'aqua',
    "volume": 'gray'
  }
      f_labels = {
    "others": 'white',
    "shipper": 'blue',
    "consignee": 'pink',
    "notify": 'orange',
    "receipt": 'purple',
    "portofloading": 'green',
    "finaldestination": 'maroon',
    "payable": 'black',
    "discharge": 'lime',
    "description": 'teal',
    "weight": 'aqua',
    "volume": 'gray'
  }

      json_df = []

      for ix, (prediction, box) in enumerate(zip(final_preds, final_bbox)):
        predicted_label = iob_to_label(prediction).lower()
        draw.rectangle(box, outline=label2color[predicted_label])
        draw.text((box[0]+10, box[1]-10), text=predicted_label, fill=label2color[predicted_label], font=font)

        json_dict = {}
        json_dict['TEXT'] = l_words[ix]
        json_dict['LABEL'] = f_labels[predicted_label]
        
        json_df.append(json_dict)
      # print('json_df',json_df)
      return image, json_df


def process_form(preds, words):
  #track the similar labels together and store indices in tracker_label_idx
  tracker_label_idx = {}
  for i in range(len(preds)):
    if preds[i] == 'others':
      pass
    elif preds[i] in tracker_label_idx:
      tracker_label_idx[preds[i]].append(i)
    else:
      tracker_label_idx[preds[i]] = [i]
  
  #update the indices with values from words
  tracker_label = {}
  for i in tracker_label_idx:
    w = ''
    for index in tracker_label_idx[i]:
      w = w +' '+ words[index]
    tracker_label[i] = w
  
  # store the unique labels for key and val
  uniq_labels =list(tracker_label.keys())
  key_ = []
  val_ = []
  for i in uniq_labels:
    if 'key' in i:
      key_.append(i)
    elif 'value' in i:
      val_.append(i)
  
  #sorting the keys and values obtained 
  key_.sort()
  val_.sort()
  # using SequenceMatcher to compare the textual key and values
  from difflib import SequenceMatcher
  def similar(a, b):
      return SequenceMatcher(None, a, b).ratio()
  pairs = {}
  for k in key_:
      pairs[k]= []
      for v in val_:
        if similar(k[:-4],v[:-6])>0.98: #here 0.7 acts as threshold for matching the strings
          pairs[k].append(v)
          break # we break as soon as match is found to reduce complexity; and since elements are sorted we are confident about the append
          
  #using pairs_ to save the key and values in form of dictonary
  pairs_ = {}
  for i in pairs:
    v = pairs.get(i)
    if len(v)!= 0:
      pairs_[tracker_label.get(i)]=[tracker_label.get(v[0])]
    else:
      pairs_[tracker_label.get(i)] = []
  
  # conversion of dict pairs_ into dataframe
  import pandas as pd
  return pd.DataFrame(pairs_.items(), columns = ['keys' ,'values'])
  
def completepreprocess(image):
  # gets the bounding boxes, predictions, extracted words and image processed
  image = image.convert("RGB")
  bbox, preds, words, image = process_image(image)

  # returns image and extracted key-value pairs along with title as the output
  im, df = visualize_image(bbox, preds, words, image)

  # process and returns k-v pairs by concatenating relevant strings.
  df_main = process_form(preds, words)

  return im,df_main

title = "Eurocorporation information extraction using LayoutLMv3 model"
description = "Eurocorporation information extraction - We use Microsoft's LayoutLMv3 trained on Eurocorp Dataset to predict the produttore_key, produttore_value, unitloc_key, unitloc_value, cliente_key, cliente_value, operatore_key, operatore_value, referente_key, referente_value, telefono_key, telefono_value, cfproduttore_key, cfproduttore_value, emailcliente_key, emailcliente_value, datarichiesta_key, datarichiesta_value, orariorichiesta_key, orariorichiesta_value,emailproduttore_key,emailproduttore_value,mattina_key,mattina_value,pomeriggio_key,pomeriggio_value,cer_key,cer_value,descrizione_key,descrizione_value,sf_key,sf_value,classpericolo_key,destino_key,destino_value,confezionamento_key,confezionamento_value,destinazione_key and destinazione_value. Results will show up in a few seconds."

article="<b>References</b><br>[1] Y. Xu et al., “LayoutLMv3: Pre-training for Document AI with Unified Text and Image Masking.” 2022. <a href='https://arxiv.org/abs/2204.08387'>Paper Link</a><br>[2]  <a href='https://github.com/NielsRogge/Transformers-Tutorials/tree/master/LayoutLMv3'>LayoutLMv3 training and inference</a><br>[3] Hongbin Sun, Zhanghui Kuang, Xiaoyu Yue, Chenhao Lin, and Wayne Zhang. 2021. Spatial Dual-Modality Graph Reasoning for Key Information Extraction. arXiv. DOI:https://doi.org/10.48550/ARXIV.2103.14470  <a href='https://doi.org/10.48550/ARXIV.2103.14470'>Paper Link</a>" 
css = """.output_image, .input_image {height: 600px !important}"""


iface = gr.Interface(fn=completepreprocess, 
                     inputs=gr.inputs.Image(type="pil",optional=True,label="upload file"), 
                    #  inputs=gr.inputs.File(file_count="multiple", label="upload file"),
                     outputs=[gr.outputs.Image(type="pil", label="annotated image"),"dataframe"] , 
                    #  outputs=,            
                     title=title,
                     description=description,
                     css=css,
                     article=article,
                     analytics_enabled = True, enable_queue=True)

iface.launch(inline=False, share=True, debug=True)
